Initial population

In [57]:
import numpy as np

In [58]:
def init_pop(pop_size):
    return np.random.randint(8, size=(pop_size,8))  #for returning some random integare num عدد العناصر ف كل صف 8 عناصر وعدد الصفوف هو السايز اللي انا هدخله

In [59]:
initial_population = init_pop(4)
print(initial_population)

[[5 2 3 4 5 1 0 1]
 [4 1 4 3 6 1 7 4]
 [2 1 7 7 2 5 1 7]
 [0 7 5 3 0 3 6 2]]


# Calculate fitness

In [60]:
def calc_fitness(population):
    fitness_vals = [] #list to save  fitness value in
    for x in population:
        penalty = 0 #penalty mean abelety to eat other queens 'the negative value of fitness' (large penalty = bad solution = low fitness)
        for i in range(8): #range = 8 cuz i have 8 vae in each row
            r = x[i] #important meanings '(r) is row of queen(i),(d) is the distance between column(i) and column (j)'
            for j in range(8):
                if i ==j:
                    continue
                d = abs(i-j) #distance = absolyet of i 'where queen stand' and j 'where i chick'
                if x[j] in [r, r-d, r+d]: # if place of queen in r or r-d or r+d then add 1 to penalty
                    penalty +=1
        fitness_vals.append(penalty) #adding values of fitness to list 'fitness_vals'
    return -1*np.array(fitness_vals) #now to return fitness i multiply penalty to -1 
    

Checking fitness function

In [61]:
initial_population = init_pop(4)
print(initial_population)

[[5 4 3 7 3 7 0 6]
 [0 2 0 5 7 3 4 1]
 [2 3 2 5 0 0 5 0]
 [0 6 3 4 6 2 2 1]]


In [62]:
fitness_vals = calc_fitness(initial_population)
print(fitness_vals)

[-12 -14 -22 -14]


# Selection
-in selection phase we  select N parents(N solution) that with higher fitness 
-Note selection is with replacement 'that means some parent can be selscted multipul times'

In [71]:
def selection(population, fitness_vals):
    probs = fitness_vals.copy()
    probs += abs(probs.min()) + 1 #عايز اعمل نسب مئويه لقياس الفيتنس ف بستخدم معادله ان كل قيمه اجمع عليها اقل قيمه +1 بحط الواحد عشان اضمن ان مفيش اي اصفار
    probs = probs/probs.sum() #هنا بجيب احتمال كل واحد فيهم عن طريقه قسمه قيمته ع عدد القيم
    N = len(population) #N must be even num  so i can apply crossover
    indices = np.arange(N)
    selected_indices = np.random.choice(indices, size=N, p=probs) #this is the probability to randomly select N parents
    selected_population = population[selected_indices]
    return selected_population

In [64]:
selection = selection(initial_population,fitness_vals )
print(selection)

[[0 2 0 5 7 3 4 1]
 [0 6 3 4 6 2 2 1]
 [5 4 3 7 3 7 0 6]
 [5 4 3 7 3 7 0 6]]


# Crossover
-after selecting N parent we  apply crossover between each two parents
-but sometimes crossover dosent happen cuz it depend on probability of random num

In [65]:
def crossover(parent1, parent2, pc): #cross over takes two selected parents and bring two childeren
    r = np.random.random() #now i make random num between 0,1 to chick prop of crossover so i make it or no
    if r <pc: #pc is probability of crossover
        m = np.random.randint(1,8) #m is place where i make crossover 'stats from index 1 and end at 7'
        child1 = np.concatenate([parent1[:m], parent2[m:]])
        child2 = np.concatenate([parent2[:m], parent1[m:]])
    else: #if r >pc teturn parents again into childeren
        child1 = parent1.copy()
        child2 = parent2.copy()
    return child1, child2

Testing crossover function

In [66]:
#parent1 = selection[0]
#parent2 = selection[1]
#child1, child2 = crossover(parent1, parent2, pc=0.70)
#print(parent1, '-->', child1)
#print(parent2, '-->', child2)

# Mutation
-randomly modify one value of an individual 'small probability'

In [67]:
def mutation(individual, pm):#mutation only take on child of crossover #pm is prbability of mutation
    r = np.random.random() #just random value to chick probability of mutation
    if r < pm:
        m = np.random.randint(8) #here i choose random index
        individual[m] = np.random.randint(8) #here i choose random num to but in choosing index
    return individual

# Applying crossover & mutation in one function

In [68]:
def crossover_mutation(selected_pop , pc, pm):
    N = len(selected_pop )
    new_pop = np.empty((N,8), dtype=int)
    for i in range (0,N,2): #step=2 cuz i have two parent in a time
        parent1 = selected_pop[i]
        parent2 = selected_pop[i+1]
        child1, child2 = crossover(parent1, parent2, pc) #applying crossover on parents
        new_pop[i] = child1  
        new_pop[i+1] = child2
    for i in range(N):
        mutation(new_pop[i], pm) #applying mutation on childeren
    return new_pop

# Putting it all together

In [72]:
def eight_queens(pop_size, max_generations, pc=0.7, pm=0.01):
    population = init_pop(pop_size)
    best_fitness_overall = None #hold best value of fitness in every generation untill reach 0
    for i_gen in range(max_generations): #chicking tonreach best fitness
        fitness_vals = calc_fitness(population)
        best_i = fitness_vals.argmax() #return index of best fitness 'argmax is built in function retuen index of highest fitness '
        best_fitness = fitness_vals[best_i] #here i store best fitness
        if best_fitness_overall is None or best_fitness > best_fitness_overall: #to put best fitness into best fitness overall if i have better solution
            best_fitness_overall = best_fitness
            best_solution = population[best_i] #best population is when fitness =0
        print(f'\ri_gen ={i_gen:06}  -f={-best_fitness_overall:03}', end='')  #i_gen print into 6 indexs  and best_fitness_overall*- to get positive value of it and it printed in  3 indexes
        if best_fitness ==0: #if i reach best solution'0 or ended without reach then then break and print solution you have
            print('\nFound optimal solution')
            break
        best_selection = selection(population, fitness_vals)
        population = crossover_mutation(best_selection, pc, pm)
    print()
    print(best_solution)

In [75]:
eight_queens(pop_size = 100, max_generations = 10000, pc=0.7, pm=0.01)

i_gen =009999  -f=002
[7 4 1 5 0 6 3 5]
